# **Data preparation**

## **1. Data cleaning**

In this step, we'll prepare the data we got from webscraping flashscore and clean it so it gets used in our machine learning model.

In [195]:
# Importing our beloved pandas ʕ•ᴥ•ʔ

import pandas as pd

Then getting our data to our notebook


In [196]:
# Importing the files to the notebook

from google.colab import files
uploaded = files.upload()

Making our csv files into dataframes:

In [197]:
# Starting with La Liga
df_li20 = pd.read_csv('Liga19-20.csv')
df_li21 = pd.read_csv('Liga20-21.csv')
df_li22 = pd.read_csv('Liga21-22.csv')
df_li23 = pd.read_csv('Liga22-23.csv')
df_li24 = pd.read_csv('Liga23-24.csv')

# Ligue 1
df_l121 = pd.read_csv('Ligue1_20-21.csv')
df_l122 = pd.read_csv('Ligue1_21-22.csv')
df_l123 = pd.read_csv('Ligue1_22-23.csv')
df_l124 = pd.read_csv('Ligue1_23-24.csv')

# Premier League
df_pl20 = pd.read_csv('PL19-20.csv')
df_pl21 = pd.read_csv('PL20-21.csv')
df_pl22 = pd.read_csv('PL21-22.csv')
df_pl23 = pd.read_csv('PL22-23.csv')
df_pl24 = pd.read_csv('PL23-24.csv')

# Bundesliga
df_bu20 = pd.read_csv('Bundes19-20.csv')
df_bu21 = pd.read_csv('Bundes20-21.csv')
df_bu22 = pd.read_csv('Bundes21-22.csv')
df_bu23 = pd.read_csv('Bundes22-23.csv')
df_bu24 = pd.read_csv('Bundes23-24.csv')

# Serie A
df_sa20 = pd.read_csv('SerieA19-20.csv')
df_sa21 = pd.read_csv('SerieA20-21.csv')
df_sa22 = pd.read_csv('SerieA21-22.csv')
df_sa23 = pd.read_csv('SerieA22-23.csv')
df_sa24 = pd.read_csv('SerieA23-24.csv')

# Primeira Liga
df_pr20 = pd.read_csv('PR19-20.csv')
df_pr21 = pd.read_csv('PR20-21.csv')
df_pr22 = pd.read_csv('PR21-22.csv')
df_pr23 = pd.read_csv('PR22-23.csv')
df_pr24 = pd.read_csv('PR23-24.csv')

In [198]:
# And store them in a list so the group manipulation gets easier:

seasons_dict = {
    'l121': df_l121, 'l122': df_l122, 'l123': df_l123, 'l124': df_l124,
    'li20': df_li20, 'li21': df_li21, 'li22': df_li22, 'li23': df_li23, 'li24': df_li24,
    'pl20': df_pl20, 'pl21': df_pl21, 'pl22': df_pl22, 'pl23': df_pl23, 'pl24': df_pl24,
    'bu20': df_bu20, 'bu21': df_bu21, 'bu22': df_bu22, 'bu23': df_bu23, 'bu24': df_bu24,
    'sa20': df_sa20, 'sa21': df_sa21, 'sa22': df_sa22, 'sa23': df_sa23, 'sa24': df_sa24,
    'pr20': df_pr20, 'pr21': df_pr21, 'pr22': df_pr22, 'pr23': df_pr23, 'pr24': df_pr24
                }

In [199]:
# Checking how many null value do we have

for season in seasons_dict:
   print(f"{season} null values: {seasons_dict[season].isnull().sum()}\n\n")

l121 null values: Unnamed: 0         0
Home team name     0
Away team name     0
Outcome            0
Home score         0
Away score         0
Home rating        0
Away rating        0
Home odds          7
Draw odds          6
Away odds          5
Home xG            4
Away xG            2
Home possession    2
Away possession    2
Home lineups       0
Away lineups       0
Home Formation     0
Away formation     0
dtype: int64


l122 null values: Unnamed: 0          0
Home team name      0
Away team name      0
Outcome             0
Home score          0
Away score          0
Home rating         0
Away rating         0
Home odds          12
Draw odds          11
Away odds          11
Home xG            10
Away xG             7
Home possession     8
Away possession     5
Home lineups        0
Away lineups        0
Home Formation      0
Away formation      0
dtype: int64


l123 null values: Unnamed: 0         0
Home team name     0
Away team name     0
Outcome            0
Home score     

We can clearly notice that some data columns are swapped, that's why we'll do our own little shuffling so we fix that, and also notice that it has taken sometimes the shots on target instead of xG, but that's ok for our first training model.

To make our life easier, let's create a function that fixes our swapped data problem:

In [200]:
def swap_xg_and_possession(df):
    """
    This function checks if the 'Home xG' or 'Away xG' columns contain a percentage.
    If so, it swaps the cell value with the corresponding 'Home possession' or 'Away possession' column.
    """

    for index, row in df.iterrows():
        # Check for '%' in 'Home xG' and swap with 'Home possession' if needed
        if isinstance(row['Home xG'], str) and '%' in row['Home xG']:
            # Only swap if both columns are not NaN or empty
            if pd.notna(row['Home possession']) and pd.notna(row['Home xG']):
                home_xg, home_possession = row['Home xG'], row['Home possession']
                df.at[index, 'Home xG'] = home_possession
                df.at[index, 'Home possession'] = home_xg

        # Check for '%' in 'Away xG' and swap with 'Away possession' if needed
        if isinstance(row['Away xG'], str) and '%' in row['Away xG']:
            # Only swap if both columns are not NaN or empty
            if pd.notna(row['Away possession']) and pd.notna(row['Away xG']):
                away_xg, away_possession = row['Away xG'], row['Away possession']
                df.at[index, 'Away xG'] = away_possession
                df.at[index, 'Away possession'] = away_xg

    return df

And now let's apply it on our dataframes:

In [201]:
for s, season in seasons_dict.items():
  swap_xg_and_possession(season)

<ipython-input-200-5059bbf70b1a>:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '43%' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Home possession'] = home_xg
<ipython-input-200-5059bbf70b1a>:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '57%' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Away possession'] = away_xg
<ipython-input-200-5059bbf70b1a>:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '75%' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Home possession'] = home_xg
<ipython-input-200-5059bbf70b1a>:22: FutureWarning: Setting an item of incompatible dtyp

And now to turn all our values numerical so they can be converted to tensors:

In [202]:
# Let's just turn the columns with string to numerical values:

for season in seasons_dict:
  seasons_dict[season]['Home possession'] = seasons_dict[season]['Home possession'].str.rstrip('%').astype(float) / 100
  seasons_dict[season]['Away possession'] = seasons_dict[season]['Away possession'].str.rstrip('%').astype(float) / 100

## **2. Features engineering**

After cleaning our data, we need to add some functionalities and organise the features in separate dataframes

### **a. Separating our dataframes**


In [203]:
results_dict = {} # We'll store only the teams names and results to create our rankings feature
ratings_dict = {} # And make this one to try to find a model based only on the ratings feature, so it has to have also the odds
xg_dict = {} # Same thing but for the xG

for season in seasons_dict:
  results_dict[f"{season}_results"] = seasons_dict[season][['Home team name', 'Away team name', 'Home score', 'Away score']].copy()
  ratings_dict[f"{season}_ratings"] = seasons_dict[season][['Home team name', 'Away team name', 'Home score', 'Away score', 'Home rating',
                                               'Away rating', 'Home odds', 'Draw odds', 'Away odds']].copy()
  xg_dict[f"{season}_xG"] = seasons_dict[season][['Home team name', 'Away team name', 'Home score', 'Away score', 'Home xG',
                                               'Away xG', 'Home odds', 'Draw odds', 'Away odds']].copy()

len(results_dict), len(ratings_dict), len(xg_dict)

(29, 29, 29)

### **b. Rankings feature**

This one is kinda tricky, it requires us to make a function that calculates the rankings:

In [204]:
def calculate_football_rankings(df):
    """
    Calculate rankings for football teams based on match results.
    Returns an empty DataFrame with correct columns if no matches are provided.
    """
    # If the DataFrame is empty, return an empty DataFrame with the correct columns
    if len(df) == 0:
        return pd.DataFrame(columns=["Team", "Points", "Goal Difference", "Rank"])

    # Initialize a dictionary to store team stats
    team_stats = {}

    # Process each match
    for _, row in df.iterrows():
        home_team = row["Home team name"]
        away_team = row["Away team name"]
        home_score = row["Home score"]
        away_score = row["Away score"]

        # Initialize stats for teams if not already present
        if home_team not in team_stats:
            team_stats[home_team] = {"Points": 0, "Goal Difference": 0}
        if away_team not in team_stats:
            team_stats[away_team] = {"Points": 0, "Goal Difference": 0}

        # Calculate goal difference for the match
        home_goal_diff = home_score - away_score
        away_goal_diff = away_score - home_score

        # Update goal differences
        team_stats[home_team]["Goal Difference"] += home_goal_diff
        team_stats[away_team]["Goal Difference"] += away_goal_diff

        # Update points based on match results
        if home_score > away_score:  # Home team wins
            team_stats[home_team]["Points"] += 3
        elif home_score < away_score:  # Away team wins
            team_stats[away_team]["Points"] += 3
        else:  # Draw
            team_stats[home_team]["Points"] += 1
            team_stats[away_team]["Points"] += 1

    # Convert the team_stats dictionary into a DataFrame
    rankings_df = pd.DataFrame([
        {"Team": team, "Points": stats["Points"], "Goal Difference": stats["Goal Difference"]}
        for team, stats in team_stats.items()
    ])

    # Sort by points, then goal difference
    rankings_df = rankings_df.sort_values(
        by=["Points", "Goal Difference"],
        ascending=[False, False]
    ).reset_index(drop=True)

    # Add rankings
    rankings_df["Rank"] = rankings_df.index + 1

    return rankings_df

And one that applies it to each round:

In [191]:
def add_current_rankings(df):
    """
    Add the current rankings to each row, based on matches up to that round.
    Handles empty DataFrames gracefully.
    """
    home_ranking = []
    away_ranking = []

    for i in range(len(df)):
        # Subset the DataFrame up to the current round
        subset_df = df.iloc[:i]

        # Calculate rankings for the current round
        current_rankings = calculate_football_rankings(subset_df)

        if len(current_rankings) == 0:
            # For the first match, both teams start unranked
            home_ranking.append(None)
            away_ranking.append(None)
        else:
            # Map rankings to teams
            ranking_dict = dict(zip(current_rankings["Team"], current_rankings["Rank"]))

            # Get rankings for the home and away teams in the current row
            home_team_rank = ranking_dict.get(df.iloc[i]["Home team name"], None)
            away_team_rank = ranking_dict.get(df.iloc[i]["Away team name"], None)

            home_ranking.append(home_team_rank)
            away_ranking.append(away_team_rank)

    # Add the rankings to the original DataFrame
    df["Home ranking"] = home_ranking
    df["Away ranking"] = away_ranking

    return df

And now to add them to all our our results dictionary dataframes:

In [214]:
# Apply the function to the results dict and then (because of its high complexity) just copy it to the other dataframes

for season in results_dict:
  results_dict[season] = add_current_rankings(results_dict[season])

results_dict['bu20_results']

,Home team name,Away team name,Home score,Away score,Home ranking,Away ranking
0,Bayern Munich,Hertha Berlin,2,2,NaN,NaN
1,Wolfsburg,FC Koln,2,1,NaN,NaN
2,Werder Bremen,Dusseldorf,1,3,NaN,NaN
3,Freiburg,Mainz,3,0,NaN,NaN
4,Dortmund,Augsburg,5,1,NaN,NaN
...,...,...,...,...,...,...
303,Bayer Leverkusen,Mainz,1,0,5.0,13.0
304,B. Monchengladbach,Hertha Berlin,2,1,5.0,10.0
305,Augsburg,RB Leipzig,1,2,14.0,4.0
306,Werder Bremen,Heidenheim,0,0,16.0,NaN


Copy this feature to the dataframes in the other dictionaries

In [217]:
#
for season in seasons_dict:
  ratings_dict[f"{season}_ratings"]['Home ranking'] = results_dict[f"{season}_results"]['Home ranking'].copy()
  ratings_dict[f"{season}_ratings"]['Away ranking'] = results_dict[f"{season}_results"]['Away ranking'].copy()

for season in seasons_dict:
  xg_dict[f"{season}_xG"]['Home ranking'] = results_dict[f"{season}_results"]['Home ranking'].copy()
  xg_dict[f"{season}_xG"]['Away ranking'] = results_dict[f"{season}_results"]['Away ranking'].copy()

ratings_dict['bu20_ratings'], xg_dict['sa24_xG']

(         Home team name Away team name  Home score  Away score  Home rating  \
 0         Bayern Munich  Hertha Berlin           2           2          6.1   
 1             Wolfsburg        FC Koln           2           1          6.4   
 2         Werder Bremen     Dusseldorf           1           3          6.4   
 3              Freiburg          Mainz           3           0          7.9   
 4              Dortmund       Augsburg           5           1          6.3   
 ..                  ...            ...         ...         ...          ...   
 303    Bayer Leverkusen          Mainz           1           0          6.9   
 304  B. Monchengladbach  Hertha Berlin           2           1          6.5   
 305            Augsburg     RB Leipzig           1           2          6.3   
 306       Werder Bremen     Heidenheim           0           0          6.7   
 307          Heidenheim  Werder Bremen           2           2          7.7   
 
      Away rating  Home odds  Draw odd

We'll quickly fill the missing rankings

In [220]:
# We'll put the rankings of unranked teams to 20 (so when a team plays playoffs gets placed last)

for season in results_dict.values():
  season[['Home ranking', 'Away ranking']] = season[['Home ranking', 'Away ranking']].fillna(20)

for season in ratings_dict.values():
  season[['Home ranking', 'Away ranking']] = season[['Home ranking', 'Away ranking']].fillna(20)

for season in xg_dict.values():
  season[['Home ranking', 'Away ranking']] = season[['Home ranking', 'Away ranking']].fillna(20)

### **c. Removing missing values**

And finally we can drop the missing values rows, because there were needed only for the rankings feature.

In [222]:
# Let's just drop'em at once

for season in ratings_dict:
    ratings_dict[season].dropna(inplace=True)
for season in xg_dict:
    xg_dict[season].dropna(inplace=True)

### **d. Last 5 matches rating feature**

In this section, we'll implement the last five matches ratings feature, that gives the average of a teams rating during its last five matches.

For that, we'll make a function that calculates the last 5 matches ratings

In [225]:
def calculate_last_five_ratings(df):
    """
    Calculate the average ratings for the last matches for each team.
    - Use average of available matches (1, 2, 3, 4, or 5).
    - If 0 matches, return 0.
    """

    # Initialize rolling lists for each team
    team_ratings = {}

    # Columns to store average ratings
    home_avg_ratings = []
    away_avg_ratings = []

    # Iterate over each match row
    for _, row in df.iterrows():
        home_team = row["Home team name"]
        away_team = row["Away team name"]
        home_rating = row["Home rating"]
        away_rating = row["Away rating"]

        # Ensure teams have their rolling lists
        if home_team not in team_ratings:
            team_ratings[home_team] = []
        if away_team not in team_ratings:
            team_ratings[away_team] = []

        # Calculate rolling average for home team
        num_matches_home = len(team_ratings[home_team])
        if num_matches_home > 0:
            last_n_matches = min(num_matches_home, 5)  # Use available matches (up to 5)
            last_avg_home = sum(team_ratings[home_team][-last_n_matches:]) / last_n_matches
        else:
            last_avg_home = 0  # No matches yet
        home_avg_ratings.append(last_avg_home)

        # Calculate rolling average for away team
        num_matches_away = len(team_ratings[away_team])
        if num_matches_away > 0:
            last_n_matches = min(num_matches_away, 5)  # Use available matches (up to 5)
            last_avg_away = sum(team_ratings[away_team][-last_n_matches:]) / last_n_matches
        else:
            last_avg_away = 0  # No matches yet
        away_avg_ratings.append(last_avg_away)

        # Update rolling lists with current ratings
        team_ratings[home_team].append(home_rating)
        team_ratings[away_team].append(away_rating)

    # Add calculated averages to the DataFrame
    df["Home Last Avg Rating"] = home_avg_ratings
    df["Away Last Avg Rating"] = away_avg_ratings

    return df


And apply it to our dataframes in ratings dictionary

In [229]:
for season in ratings_dict:
  ratings_dict[season] = calculate_last_five_ratings(ratings_dict[season])

## **3. Saving our data**

Last but not least, we'll save our dataframes to csv files so they get used for the machine learning step of our ambitious project.

In [232]:
for season in results_dict:
  results_dict[season].to_csv(f'results/{season}.csv', index=False)

for season in ratings_dict:
  ratings_dict[season].to_csv(f'ratings/{season}.csv', index=False)

for season in xg_dict:
  xg_dict[season].to_csv(f'xg/{season}.csv', index=False)

And download them to our local machine

In [235]:
import shutil
from google.colab import files

shutil.make_archive("ratings", 'zip', "ratings")
files.download("ratings.zip")

shutil.make_archive("results", 'zip', "results")
files.download("results.zip")

shutil.make_archive("xg", 'zip', "xg")
files.download("xg.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Thank you for your attention !